---------------------------------------------
Sloučení dat z PPL, OX-Balíkovna, DPD, DHL,  
Penguin (pro finální merge s 1.setem)
---------------------------------------------
Tento skript:
- Načte 4 datasety (PPL, OX-Balíkovna, DPD, DHL, Penguin)
- Zafixuje datové typy dle požadavků
- Sloučí je pod sebe (concat)
- Vytvoří nový jednotný ID sloupec
- Manuálně dočistí chybné údaje
- Sjednotí označení chybějících hodnot
- Zajistí kompatibilitu s SQL databází, resp. s powerBI

In [106]:
import pandas as pd

# Pomocná funkce pro načtení a očištění datasetu

def load_dataset(path, source_name):
    df = pd.read_csv(path, encoding='utf-8')
    df.columns = df.columns.str.strip()

    # Oprava typů a čištění hodnot
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longtitude'] = pd.to_numeric(df['longtitude'], errors='coerce')
    df['name'] = df['name'].astype(str)
    df['city'] = df['city'].astype(str)
    df['zip_code'] = df['zip_code'].astype(str)
    df['house_NO'] = df['house_NO'].astype(str) #je třeba ještě dočistit 
    df['street'] = df['street'].astype(str)
    df['source'] = source_name
    # odstranění desetinných, tam kde vznikly
    df['city_code'] = df['city_code'].apply(lambda x: str(x).split('.')[0])
    return df

# Načtení datasetů
ppl = load_dataset("C:/Users/Sabina/Sabi dokumenty/01 Czechitas/01 PYTHON/Python/Detektivky_vizualni_smog/sabi/data/Hotovo_unikatni_ppl_adresy_kod_obce.csv", "ppl")
OX = load_dataset("C:/Users/Sabina/Sabi dokumenty/01 Czechitas/01 PYTHON/Python/Detektivky_vizualni_smog/sabi/data/box_ox_enriched - box_ox_enriched.csv", "ox")
DPD = load_dataset("C:/Users/Sabina/Sabi dokumenty/01 Czechitas/01 PYTHON/Python/Detektivky_vizualni_smog/sabi/data/lf_clean_data_dpd.csv", "dpd")
DHL = load_dataset("C:/Users/Sabina/Sabi dokumenty/01 Czechitas/01 PYTHON/Python/Detektivky_vizualni_smog/sabi/data/lf_clean_data_dhl.csv", "dhl")
Penguin = load_dataset("C:/Users/Sabina/Sabi dokumenty/01 Czechitas/01 PYTHON/Python/Detektivky_vizualni_smog/sabi/data/penguin_enriched.csv", "penguin")

# Ruční oprava chybných hodnot ve sloupci 'house_NO'
OX['house_NO'] = OX['house_NO'].apply(lambda x: x[:-2] if isinstance(x, str) and x.endswith('.0') else x)

# Spojení všech datasetů
dfs = [ppl, OX, DPD, DHL, Penguin]
combined_df = pd.concat(dfs, ignore_index=True)

# Přidání unikátního ID
combined_df['id'] = range(1, len(combined_df) + 1)

# Sjednocení sloupců a jejich pořadí
ordered_cols = ['id', 'name', 'city_code', 'city', 'zip_code', 'street', 'house_NO', 'latitude', 'longtitude', 'source']
combined_df = combined_df[ordered_cols]

# Ukázka prvních řádků výsledku
combined_df.head()

    # city_code a house_NO jako string + odstranění desetinných míst pokud vznikly
    

,id,name,city_code,city,zip_code,street,house_NO,latitude,longtitude,source
0,1,PPL ParcelBox,597635,Město Albrechtice,79395,Míru,nan,50.162559,17.573313,ppl
1,2,PPL ParcelBox,598925,Albrechtice,73543,Středová,408,49.783430,18.522770,ppl
2,3,PPL ParcelBox,581291,Adamov,67904,Nádražní,42,49.300471,16.651568,ppl
3,4,PPL ParcelBox,549258,Albrechtice nad Vltavou,39816,Albrechtice nad Vltavou,16,49.256399,14.307878,ppl
4,5,PPL ParcelBox,554499,Aš,35201,Chebská,2934,50.213569,12.199983,ppl


5. Kontrola datových typů 

In [107]:
print(combined_df.dtypes)

id              int64
name           object
city_code      object
city           object
zip_code       object
street         object
house_NO       object
latitude      float64
longtitude    float64
source         object
dtype: object


Úprava PPL 
je už ve zdrojovém souboru, musím risknout tuto opravud namátkově to vychází
1933-04-01 0:00:00  
mělo být 1933/4  

In [108]:
ppl_problem = ppl[ppl['house_NO'].str.contains('00:00:00', na=False)]

ppl['house_NO'].str.contains('00:00:00', na=False).sum()

np.int64(51)

In [109]:
# 1. Odstraníme vše za poslední pomlčkou (včetně ní)
ppl['house_NO'] = ppl['house_NO'].str.replace(r'-[^-]*$', '', regex=True)

# 2. Zaměníme zbývající pomlčky za lomítka
ppl['house_NO'] = ppl['house_NO'].str.replace('-', '/', regex=False)

# 3. Odstraníme úvodní nulu, pokud je na začátku
ppl['house_NO'] = ppl['house_NO'].str.lstrip('0')

In [110]:
ppl['house_NO'].str.contains('00:00:00', na=False).sum()

np.int64(0)

Manuální úpravy - doplnění house_no, kde bylo chybně psč

In [111]:
# Slovník: id → nová hodnota house_NO
opravit_house_no = {
    1515: "1934/8",
    594: "nan",
    978: "1104/35",
    882: "878/6",
}

# Aplikace změn
for id_, new_value in opravit_house_no.items():
    combined_df.loc[combined_df['id'] == id_, 'house_NO'] = new_value

# Kontrola změn
combined_df[combined_df['id'].isin(opravit_house_no.keys())][['id', 'street', 'house_NO']]

,id,street,house_NO
593,594,Buzulucká,nan
881,882,Riegrova,878/6
977,978,Starohorská,1104/35
1514,1515,Nádražní,1934/8


Manuální úpravy - záměna "-"  
někde bylo nan zapsáno pomlčkou
řeším ručně u daných řádků, nechci si pokazit zbylé hodnoty  
nemám už čas na další exploraci

In [112]:
import numpy as np

# Seznam ID, kterých se to týká
ids_na_house_no = [2191, 2209, 2248, 2256, 2269, 2275, 2331, 2356, 2379, 2418, 2420, 2259]

# Nahrazení pomlčky nebo podtržítka za NaN pouze u vybraných ID
combined_df.loc[
    (combined_df['id'].isin(ids_na_house_no)) & 
    (combined_df['house_NO'].isin(['-', '_'])),
    'house_NO'
] = np.nan

# Kontrola změn
combined_df[combined_df['id'].isin(ids_na_house_no)][['id', 'house_NO']]

,id,house_NO
2190,2191,NaN
2208,2209,NaN
2247,2248,NaN
2255,2256,NaN
2258,2259,NaN
2268,2269,NaN
2274,2275,NaN
2330,2331,NaN
2355,2356,NaN
2378,2379,NaN


In [113]:
# Oprava pro id 5185
combined_df.loc[combined_df['id'] == 5185, 'house_NO'] = None
combined_df.loc[combined_df['id'] == 5185, 'street'] = "9. května"

# Volitelná kontrola
combined_df[combined_df['id'] == 5185][['id', 'street', 'house_NO']]

,id,street,house_NO
5184,5185,9. května,None


Je třeba sjednotit prázdné řádky, (string "nan") na skutečné NaN

In [114]:
combined_df['house_NO'] = combined_df['house_NO'].replace(["0", "", " ", "nan", None], pd.NA)

In [115]:
combined_df[combined_df['house_NO'].isna()]

,id,name,city_code,city,zip_code,street,house_NO,latitude,longtitude,source
0,1,PPL ParcelBox,597635,Město Albrechtice,79395,Míru,<NA>,50.162559,17.573313,ppl
27,28,PPL ParcelBox,589268,Bedihošť,79821,Prostějovská,<NA>,49.448438,17.166101,ppl
73,74,PPL ParcelBox,585076,Biskupice,76341,Biskupice,<NA>,49.084469,17.709433,ppl
74,75,PPL ParcelBox,588393,Bystřice pod Hostýnem,76861,Školní,<NA>,49.400737,17.681587,ppl
86,87,PPL ParcelBox,540013,Březnice,26272,Březnice,<NA>,49.558704,13.946270,ppl
...,...,...,...,...,...,...,...,...,...,...
5152,5153,Penguin box,599565,Kopřivnice,nan,Lubina,<NA>,NaN,NaN,penguin
5178,5179,Penguin box,599808,Příbor,nan,Masarykova,<NA>,NaN,NaN,penguin
5179,5180,Penguin box,599808,Příbor,nan,Svatopluka Čecha,<NA>,NaN,NaN,penguin
5180,5181,Penguin box,599808,Příbor,nan,Vrchlického,<NA>,NaN,NaN,penguin


In [116]:
# Sjednocení ostatních textových hodnot na NaN (bez nahrazování "0")
cols_to_clean = [col for col in combined_df.columns if col not in ['id', 'source', 'house_NO']]
for col in cols_to_clean:
    if combined_df[col].dtype == object:
        combined_df[col] = combined_df[col].replace(["", " ", "nan", None], pd.NA)

6. Uložení finálního datasetu (např. pro SQL import)

In [117]:
# Převod jmen společností na jejich ID
spolecnost_map = {
    "GLS": 1,
    "Alza": 2,
    "Zásilkovna": 3,
    "PPL ParcelBox": 4,
    "DHL Locker": 5,
    "DPD box": 6,
    "Penguin box": 7,
    "OX - Balíkovna": 8
}
combined_df['id_spolecnost'] = combined_df['name'].map(spolecnost_map)

# Přejmenování sloupců dle dim_box
combined_df = combined_df.rename(columns={
    'id': 'id',
    'street': 'ulice',
    'house_NO': 'cislo',
    'zip_code': 'psc',
    'city': 'mesto',
    'latitude': 'lat',
    'longtitude': 'long',
    'city_code': 'kod_obec'
})

# Převod datových typů dle struktury tabulky dim_box
combined_df['cislo'] = combined_df['cislo'].astype(str)
combined_df['psc'] = combined_df['psc'].astype(str)
combined_df['kod_obec'] = combined_df['kod_obec'].astype(str)
combined_df['ulice'] = combined_df['ulice'].astype(str)
combined_df['mesto'] = combined_df['mesto'].astype(str)
combined_df['lat'] = combined_df['lat'].astype(float)
combined_df['long'] = combined_df['long'].astype(float)
combined_df['id_spolecnost'] = combined_df['id_spolecnost'].astype(int)

# Zachování pouze sloupců dle datového modelu dim_box
combined_df = combined_df[['id', 'id_spolecnost', 'kod_obec', 'psc', 'mesto', 'ulice', 'cislo', 'mesto', 'lat', 'long' ]]

# Export do CSV připravený pro Power BI
combined_df.to_csv(
    r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\dim_box2.csv',
    sep=";",
    index=False,
    quoting=1,
    quotechar='"',
    encoding="utf-8"
)